In [1381]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore
import numpy as np
import pandas as pd

train=pd.read_csv('train.csv')
y_train=train['time']

test=pd.read_csv('test.csv')

testID=test['id']
x_test=test.drop(['id'],axis=1)
x_train=train.drop(['time','id'],axis=1)

x_train['Type']='Train'
x_test['Type']='Test'
#Combine trainFeatures.csv and testFeatures.csv
fullData =pd.concat([x_train,x_test],axis=0)


In [1382]:
#fullData.describe()

In [1383]:
#fullData.isnull().any()

In [1384]:
#fullData.head(30)

In [1385]:
y_train.head(10)

0    0.409987
1    3.950953
2    0.368702
3    1.004559
4    0.802800
5    7.916113
6    2.206062
7    2.407143
8    1.002341
9    1.607144
Name: time, dtype: float64

In [1386]:
#one_hot=pd.get_dummies(fullData)
#fullData= one_hot.astype('float')

#fullData.head()

In [1387]:
#One-hot-encoding features
#cat_cols = ['alpha']

#把字符属性数字化(其实也是字符，显示的是数字，但实际是字符属性)
#for var in cat_cols:
   # one_hot=pd.get_dummies(fullData)
    #fullData= one_hot.astype('float')

   # number = LabelEncoder()
   # fullData[var] = number.fit_transform(fullData[var].astype('float'))
#fullData.head()

In [1388]:

cat_cols = ['penalty']
for var in cat_cols:
    number = LabelEncoder()
    fullData[var] = number.fit_transform(fullData[var].astype('str'))
fullData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 99
Data columns (total 14 columns):
penalty                 500 non-null int64
l1_ratio                500 non-null float64
alpha                   500 non-null float64
max_iter                500 non-null int64
random_state            500 non-null int64
n_jobs                  500 non-null int64
n_samples               500 non-null int64
n_features              500 non-null int64
n_classes               500 non-null int64
n_clusters_per_class    500 non-null int64
n_informative           500 non-null int64
flip_y                  500 non-null float64
scale                   500 non-null float64
Type                    500 non-null object
dtypes: float64(4), int64(9), object(1)
memory usage: 58.6+ KB


In [1389]:
fullData=fullData.replace({'n_jobs': -1}, 16)


In [1390]:
fullData['nn']=fullData['n_samples']*fullData['n_features']*fullData['max_iter']/fullData['n_jobs']
fullData.head()

,penalty,l1_ratio,alpha,max_iter,random_state,n_jobs,n_samples,n_features,n_classes,n_clusters_per_class,n_informative,flip_y,scale,Type,nn
0,3,0.304083,0.0001,417,475,16,1089,327,4,3,7,0.074798,24.242009,Train,9.280934e+06
1,1,0.727744,0.0010,578,569,1,790,373,4,5,7,0.077781,54.626302,Train,1.703193e+08
2,3,0.745885,0.0100,588,529,2,428,1198,2,5,6,0.030196,17.999964,Train,1.507467e+08
3,3,0.474605,0.0010,829,103,4,877,313,6,5,7,0.057261,82.257222,Train,5.689033e+07
4,0,0.395049,0.0010,167,418,2,216,644,8,5,11,0.073728,95.515601,Train,1.161518e+07


In [1391]:
#fullData['m']=fullData['n_classes']*fullData['n_clusters_per_class']

In [1392]:
fullData['n_jobs']=1/fullData['n_jobs']

In [1393]:
fullData[['l1_ratio','max_iter','random_state','n_jobs','n_classes','flip_y','nn','n_samples','n_features','n_jobs','penalty','n_clusters_per_class']] =fullData[['n_clusters_per_class','l1_ratio','max_iter','random_state','n_jobs','n_classes','flip_y','nn','n_samples','n_features','n_jobs','penalty']].apply(zscore)


In [1394]:
fullData = fullData.drop(['n_informative','alpha','scale','random_state'], axis=1)
fullData.head()

,penalty,l1_ratio,max_iter,n_jobs,n_samples,n_features,n_classes,n_clusters_per_class,flip_y,Type,nn
0,-0.947265,-0.397966,-0.869638,-1.175096,-0.603617,0.622985,-0.947265,1.333617,-0.766463,Train,0.846893
1,1.899084,1.410970,0.966533,-1.067687,0.169866,-0.059317,1.899084,-0.414243,-0.766463,Train,0.945802
2,0.381031,1.410970,1.045158,0.858677,0.075857,-0.885381,0.381031,1.333617,-1.569042,Train,-0.632018
3,-0.377995,1.410970,-0.130585,-1.207786,-0.374944,0.139212,-0.377995,1.333617,0.036116,Train,0.265404
4,0.381031,1.410970,-0.475383,-0.434905,-0.592405,-1.369153,0.381031,-1.288172,0.838695,Train,0.811406


In [1395]:
#splitting the data into x_train and x_test, and drop the column "type"
x_train=fullData.loc[fullData['Type'].isin(['Train'])].drop(['Type'], axis=1)
x_test=fullData.loc[fullData['Type'].isin(['Test'])].drop(['Type'], axis=1)
#x_train.head(35000)

In [1396]:
x_train.head(10)

,penalty,l1_ratio,max_iter,n_jobs,n_samples,n_features,n_classes,n_clusters_per_class,flip_y,nn
0,-0.947265,-0.397966,-0.869638,-1.175096,-0.603617,0.622985,-0.947265,1.333617,-0.766463,0.846893
1,1.899084,1.410970,0.966533,-1.067687,0.169866,-0.059317,1.899084,-0.414243,-0.766463,0.945802
2,0.381031,1.410970,1.045158,0.858677,0.075857,-0.885381,0.381031,1.333617,-1.569042,-0.632018
3,-0.377995,1.410970,-0.130585,-1.207786,-0.374944,0.139212,-0.377995,1.333617,0.036116,0.265404
4,0.381031,1.410970,-0.475383,-0.434905,-0.592405,-1.369153,0.381031,-1.288172,0.838695,0.811406
5,-0.377995,-1.302434,-0.632328,0.071787,0.774879,1.323542,-0.377995,-0.414243,-0.766463,1.599101
6,0.381031,-1.302434,1.100114,-0.325161,0.751623,0.209953,0.381031,1.333617,-0.365173,1.527509
7,-0.757508,1.410970,-0.961702,-0.217752,-0.521779,-0.317177,-0.757508,-0.414243,1.239984,-0.291583
8,-0.757508,0.506502,1.133098,0.842332,-0.538515,-0.725645,-0.757508,-0.414243,-1.569042,-0.783136
9,-0.757508,1.410970,-1.108289,-0.063642,-0.308884,1.287031,-0.757508,0.459687,1.239984,-0.816902


In [1398]:
from sklearn.model_selection import train_test_split
xx_train, xx_valid, yy_train, yy_valid = train_test_split(
   x_train, y_train, test_size=0.2, random_state=0)




In [1399]:
import xgboost as xgb

model_xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.23, gamma=0.07, subsample=0.768,
                           colsample_bytree=1, max_depth=3,min_child_samples=4)
          


In [1400]:
model_xgb.fit(xx_train,yy_train)

xgb_preds=model_xgb.predict(xx_valid)

param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator =xgb(learning_rate=1, max_depth=6, n_estimators=2000)
gsearch1.fit(xx_train,yy_train)
gsearch1.grid_scores_, gsearch1.best_params_,     gsearch1.best_score_


In [1401]:
from sklearn.metrics import mean_squared_error, r2_score

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(yy_valid, xgb_preds))

Mean squared error: 3.43


In [1295]:
final_class=model_xgb.predict(x_test)
final_class

array([ 1.2303171e+00,  8.7516928e+00,  1.7294779e-01,  1.4989152e+00,
        2.1498156e+00,  9.3036289e+00,  2.3495178e+00,  8.4741938e-01,
        1.1903320e+01,  3.9392552e-01,  5.4113421e+00,  8.4813881e+00,
        1.1495911e+00,  2.6776243e+01,  4.3834576e-01,  9.6211475e-01,
        1.3617823e+00,  1.0357168e+01,  4.0037184e+00,  1.6246024e+00,
        7.4893236e-03,  8.1785691e-01,  1.0795174e+00,  1.2364430e+00,
        1.4341247e+00,  1.7446536e+00,  2.2339776e+00,  2.9781826e+00,
        2.1775229e+00,  2.7604043e+00,  1.7802406e+01,  2.7712770e+00,
        7.7552468e-01,  7.0930405e+00,  7.4601965e+00,  6.7717040e-01,
        4.0987215e+00,  2.1683373e+00,  2.7473795e-01,  3.6610119e+00,
        3.3040740e+00,  8.9950228e+00,  1.6470580e+00,  4.0735502e+00,
        5.8408654e-01,  5.1760263e+00,  3.5282905e+00,  1.1202635e+00,
        1.1442137e+01,  8.8701057e-01,  1.4678955e-01,  1.5916898e+01,
       -3.7781775e-02,  6.3399863e+00,  8.0266857e+00,  7.0691395e-01,
      

In [1242]:
#For ajudsting final outputs
for i in range(0,len(final_class)):
    if final_class[i]<0:

        final_class[i]=0.07
    else:
        final_class[i]+=0.8
    continue


In [1243]:
pd.DataFrame({"id":testID,"time":final_class}).to_csv('kaggleoutputxgbV2.csv',columns=['id','time'],index=None)
                   